## Visualize collected `geospatial data` on a <span style="color:turquoise;">MAP</span>

In [8]:
import os
from dotenv import load_dotenv
load_dotenv() 
Bearer = os.getenv('Bearer')
apiurl = os.getenv('apiurl')


http://105.29.165.253:13000


In [19]:
import requests
from pprint import pprint
#Define the bldg code service function
#ONU
#Enter user number
num = int(input("Please enter a number: "))
def getONU(num):
    url = f"{apiurl}/api/komp_assistant/onu/phone/{num}"

    payload = {}
    headers = {
    'Authorization': Bearer
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        #pprint(data)
        bcode_value = data.get("bcode")
        bname_value = data.get("bname")
        return bcode_value, bname_value
    else:
        bcode_value = None
        return None

def getPAP(num):
    url = f"{apiurl}/api/komp_assistant/phone/{num}"

    payload = {}
    headers = {
    'Authorization': Bearer
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        bcode_value = data.get("bcode")
        bname_value = data.get("bname")
        return bcode_value, bname_value
    else:
        bcode_value = None
        return None

def getbcode(num):
    try:
        bcode, bname = getPAP(num)
        if bcode == None:
            bcode, bname = getONU(num)
            if bcode == None:
                print('Bcode not found for both PAP and ONU')
                return 'Not Found'
            else:
                #print('ONU', num, bcode)
                return 'ONU', num, bcode, bname
        else:
            #print('PAP', num, bcode)
            return 'PAP', num, bcode, bname
    except Exception as e:
        print('Error: ',e)
result = getbcode(num)
print(result)
if isinstance(result, tuple):
    print("Result is a tuple")
    router, num, buildingcode, bname = result
else:
    print("Result is not a tuple")
    buildingcode = 'Not Exist'


{'ap_type': 'ONU',
 'bcode': 'KENKS07260',
 'bname': 'Peter Bungalow',
 'champ': {'department': 'Sales',
           'email': 'crisarahkincy@gmail.com',
           'name': 'Sarah Gitau',
           'region': 'KSN'},
 'client': 'Owner',
 'clientid': 2655,
 'door': 'Nil',
 'floor': 0,
 'installation': {'dateinstalled': '2023-07-14',
                  'mac': 'BC-60-6B-D2-50-B0',
                  'techies': {'techies': 'Brandold Ochieng,Charles Thomas'}},
 'phone1': 719265315,
 'phone2': None,
 'region': 'STN',
 'status': 'Active',
 'turnon': {'active': 1,
            'confirmed_by': 'Joseph Masika',
            'dateturnedon': '2023-07-14',
            'gpon_no': 'gpon/14',
            'mac': 'BC-60-6B-D2-50-B0',
            'olt': 2,
            'onu_code': 'KENKS07260-001U',
            'port_number': 55,
            'serial_code': 'BDCM:6BD250B0',
            'serial_no': '0200222088192'}}
('ONU', 719265315, 'KENKS07260', 'Peter Bungalow')
Result is a tuple


In [20]:
#%pip install folium
import zipfile
import xml.etree.ElementTree as ET
import folium

with zipfile.ZipFile('./geobank/KCIS Master Map 3010_2023.kmz', 'r') as zip_ref:
    zip_ref.extractall('./geobank/')

# Parse the .kml file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

In [21]:
for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
    #print(name)

In [22]:
print(buildingcode)
buildingcode = str(buildingcode)
for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
    #print(name)
    if buildingcode in name: 
        coordinates = placemark.find('{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates').text
        # coordinates will be a string in the format "longitude,latitude,altitude"
        # you can split it into a list of floats using:
        coordinates_list = [float(coord) for coord in coordinates.split(',')]
        print(coordinates_list)


KENKS07260
[36.9171650676369, -1.236455735884236, 0.0]


In [23]:
longitude, latitude = coordinates_list[:2]
# Create a folium map centered around the coordinates
map_object = folium.Map(location=[latitude, longitude], zoom_start=15)

# Add a marker to the map
folium.Marker(location=[latitude, longitude], popup=f'Contact: {num}, Router Type: {router}, Building: {bname}').add_to(map_object)

# Save the map to an HTML file
map_object.save('./geobank/map.html')


In [24]:
import sqlite3
from pprint import pprint

# Connect to the SQLite database
conn = sqlite3.connect('transcriptions.db')
cursor = conn.cursor()

# Initialize an empty dictionary to store results
results_dict = {}

# Retrieve unique contact numbers from the transcriptions table
cursor.execute("SELECT DISTINCT contact FROM transcriptions WHERE contact IS NOT NULL;")
# cursor.execute("SELECT DISTINCT contact FROM transcriptions WHERE contact IS NOT NULL AND (substr(date, 1, 10) = '2023-10-28' AND issue_category = 'Router Technical Problem');")


contacts = cursor.fetchall()

# Iterate over each contact number
for contact in contacts:
    num = contact[0]

    # Use the getbcode function to get router, num, building code, and bname
    result = getbcode(num)

    if isinstance(result, tuple):
        router, num, buildingcode, bname = result
    else:
        buildingcode = 'Not Exist'

    # Store the results in the dictionary
    results_dict[num] = {
        'router': router,
        'num': num,
        'buildingcode': buildingcode,
        'bname': bname
    }

# Close the database connection
conn.close()

# Print the results dictionary
pprint(results_dict)


{'ap_type': 'ONU',
 'bcode': 'KENKM07054',
 'bname': 'YOGURT',
 'champ': {'department': 'Sales',
           'email': 'johnmacharisndiga@gmail.com',
           'name': 'John Macharia',
           'region': 'MWKS'},
 'client': 'Mamu',
 'clientid': 11200,
 'door': '11',
 'floor': 2,
 'installation': {'dateinstalled': '2023-10-15',
                  'mac': 'AC-12-8E-45-B0-90',
                  'techies': {'techies': 'Samuel Githua Wambui,Samuel Githua '
                                         'Wambui'}},
 'phone1': 725815945,
 'phone2': None,
 'region': 'MWKN',
 'status': 'Active',
 'turnon': {'active': 1,
            'confirmed_by': 'System Api',
            'dateturnedon': '2023-10-15',
            'gpon_no': 'gpon/15',
            'mac': 'AC-12-8E-45-B0-90',
            'olt': 3,
            'onu_code': 'KENKM07054-201U',
            'port_number': 0,
            'serial_code': 'AHDI:8E45B090',
            'serial_no': '0200222170708'}}
{'ap_type': 'ONU',
 'bcode': 'KENKS06181',
 'bna

In [25]:
import folium
from regex import B

# Initialize an empty folium map
map_object = folium.Map(location=[-1.2921, 36.8219], zoom_start=10)  # Default centre initialized Nairobi, Kenya and zoom

# Iterate over each user in the results_dict
for num, user_info in results_dict.items():
    # Extract user information
    router = user_info['router']
    buildingcode = user_info['buildingcode']
    if 'KENKH' in buildingcode:
        iconcolor = 'blue'
    elif 'KENKS' in buildingcode:
        iconcolor = 'darkblue'
    elif 'KENKM' in buildingcode:
        iconcolor = 'purple'
    elif 'KEKRK' in buildingcode:
        iconcolor = 'cadetblue'
    elif 'KENKK' in buildingcode:
        iconcolor = 'gray'
    elif 'KENRG' in buildingcode:
        iconcolor = 'black'
    elif 'KENRZ' in buildingcode:
        iconcolor = 'beige'
    elif 'KEKRG' in buildingcode:
        iconcolor = 'lightblue'
    elif 'KENRK' in buildingcode:
        iconcolor = 'green'
    elif 'KENRL' in buildingcode:
        iconcolor = 'lightgray'
    elif 'KENRR' in buildingcode:
        iconcolor = 'darkblue'
    else:
        iconcolor = 'orange'

    bname = user_info['bname']

    # Check if building code is available
    if buildingcode and ('Not Exist' not in buildingcode) and (isinstance(buildingcode, str) or isinstance(buildingcode, tuple)):
        if isinstance(buildingcode, str):
            buildingcode_list = [buildingcode]
        else:
            buildingcode_list = list(buildingcode)

        for bcode in buildingcode_list:
            # Iterate over each placemark in the KML file
            for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
                name = placemark.find('{http://www.opengis.net/kml/2.2}name').text

                # Check if the building code is in the name
                if bcode in name:
                    coordinates = placemark.find('{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates').text
                    coordinates_list = [float(coord) for coord in coordinates.split(',')]

                    # Extract latitude and longitude
                    longitude, latitude = coordinates_list[:2]

                    # Add a marker for each user to the folium map
                    folium.Marker(
                        location=[latitude, longitude],
                        popup=f'Contact: {num}, Router Type: {router}, Building: {bname}',icon=folium.Icon(color=iconcolor, icon='earphone' )
                    ).add_to(map_object)

# Save the map to an HTML file
map_object.save('./geobank/map.html')


In [1]:
import folium
from xml.etree import ElementTree as ET

def add_polygon(map_object, coordinates, name):
    # Extract latitudes, longitudes, and altitudes from the coordinates
    latitudes = [coord[1] for coord in coordinates]
    longitudes = [coord[0] for coord in coordinates]

    # Check if there are coordinates before calculating the center
    if latitudes and longitudes:
        # Find the center of the polygon
        center_lat = sum(latitudes) / len(latitudes)
        center_lon = sum(longitudes) / len(longitudes)

        # Create a Folium polygon
        folium.Polygon(
            locations=[(lon, lat) for lon, lat, _ in coordinates],  # Use all three values (lat, lon, _)
            color='blue',
            fill=True,
            fill_color='lightblue',
            fill_opacity=0.4,
            popup=name,
            location=(center_lat, center_lon)  # Set the center of the polygon
        ).add_to(map_object)

def process_folder(folder, map_object):
    for placemark in folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
        coordinates_text = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
        coordinates = [list(map(float, coord.split(','))) for coord in coordinates_text.split()]
        add_polygon(map_object, coordinates, name)

# Parse the KML file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

# Create a Folium map
map_object = folium.Map(location=[0, 0], zoom_start=3)

# Process top-level folders
for folder in root.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
    process_folder(folder, map_object)

# Save or display the map
map_object.save('./geobank/map_with_outlines.html')


### As observed the created html file is too large to render on browser without being `CPU intensive`. 
#### Solution: 
Externalize Data: Instead of embedding all data directly into the HTML file, consider loading data dynamically from an external source (e.g., GeoJSON files) as needed. This can improve initial loading times.

In [4]:
#%pip install geopandas
%pip install Fiona pyproj
import geopandas as gpd

# Read KML file into a GeoDataFrame
gdf = gpd.read_file('./geobank/doc.kml', driver='KML')

# Save GeoDataFrame as GeoJSON
gdf.to_file('./geobank/output.geojson', driver='GeoJSON')


Note: you may need to restart the kernel to use updated packages.


DriverError: unsupported driver: 'KML'